In [2]:
#%matplotlib inline
#%config InlineBackend.figure_format='retina'
#%load_ext autoreload
#%autoreload 2
# ----------------------------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import time
# ----------------------------------------------------------------------------------------------
from absopac import *
#from aerosols import *
from integrate_transits import *
from setup_atm import *
#from vis import *

ModuleNotFoundError: No module named 'integrate_transits'

In [ ]:
def tspec_from_dict(atmdict):
    '''
    Input dictionary should have 
    wlrange in microns
    beta in degrees
    Z in terms of Zsolar
    everything else with dimensions in SI units
    '''    
    nalt = atmdict['nalt']   # number of altitude grid layers
    nlong = atmdict['nlong'] # number of longitude grid points
    nlat = atmdict['nlat']   # number of latitude grid points
    atm_type = atmdict['atm_type'] # should be one of: clear1d, clear3d, aerosol1d, aerosol3d, eqCloud1d, eqCloud3d    
    
    P0 = atmdict['P0']  # reference pressure in Pascals
    Z = atmdict['Z']    # metallicity in multiples of Z_sun
    Mp = atmdict['Mp']  # Mass of the planet in kg
    R0 = atmdict['Rp']  # reference radius for planet where it is opaque in meters
    Rstar = atmdict['Rstar'] # radius of star in meters
    wlrange = atmdict['wlrange'] # wavelength array for spectrum, in microns
    nwl = len(wlrange)

    def gasOpacFunc(Z,T,P): #based on metallicity,
        # expects T in Kelvin, P in pascals
        # expects T and P to be 1d arrays
        # of equal length
        mu = mu_func(Z,T,P)
        rho = eos_idealgas(P, mu*gmol_to_kg, T)
        gas = gasopac_func(Z, wlrange, T, rho)
        rayleigh = rayleigh_func(Z, wlrange, T, rho)
        return gas + rayleigh # units are light blocked per m, has shape (nT, nwl) or (nwl,) if nz=1
    
    if atm_type == 'clear1d':
        Temp = atmdict['Temp']
        solved_atm = atm_setup_1d(Temp,Mp,R0,P0,nwl,nalt,nlong,nlat,mu_func,Z) #atmosphere set up
        Temps, Pressures, alt_range, long_range, lat_range, atmosphere_grid = solved_atm
        true_depth = effective_area_at_full_depth_no_long_dep(alt_range, long_range, 
                    lat_range, Temps, Pressures, gasOpacFunc,nwl,100,Z)/(np.pi*Rstar**2.0)
        
    else:
        print('Unknown atmosphere type, please pick one of: \n') 
        print('clear1d, clear3d, aerosol1d, aerosol3d, eqCloud1d, eqCloud3d')
        return None
    
    return true_depth

## example of computing single T-P profile clear transit spectrum (note these are just isothermal for now...)

In [ ]:
atmdict = { 'wlrange': np.logspace(np.log10(0.5),np.log10(12),300), # wavelength array for spectrum, in microns
            'nalt': 100, # number of altitude grid layers
            'nlong':60, # number of longitude grid points
            'nlat':60,  # number of latitude grid points
 
            'P0':10**5, # reference pressure in Pascals
            'Z':1.0, # metallicity in multiples of Z_sun
            'Mp': 0.8*Mj, # mass of the planet in kg
            'Rp': 1.5*Rj,  # reference radius for the planet where all wavelengths are opaque, in meters
            'Rstar': 0.9*Rsun, # radius of host star in meters    
           
            'atm_type':'clear1d', # should be one of: clear1d, clear3d, aerosol1d, aerosol3d, eqCloud1d, eqCloud3d    
            'Temp':1600, # isothermal temperature
        }
clear1d_tspec = tspec_from_dict(atmdict)
plt.semilogx(atmdict['wlrange'],clear1d_tspec*100)
plt.xlabel('Wavelength ($\mu$m)',fontsize=18)
plt.ylabel('Transit Depth (%)',fontsize=18) 

In [9]:
# ----------------------------------------------------------------------------------------------
# Data for 13 planets with estimated day and night temperatures
# taken from table of values from Keating & Cowan 2019, double check with new data?
# ----------------------------------------------------------------------------------------------
names = ['HD189733b', 'WASP-43b','HD209458b','CoRoT-2b','HD149026b','WASP-14b',
             'WASP-19b','HAT-P-7b','KELT-1b','WASP-18b','WASP-103b','WASP-12b','WASP-33b'] #waspy babies
t0 = np.array([1636,2051,2053,2175,2411,2654,2995,3211,3391,3412,3530,3636,3874]) #equillibrium temp
tday = np.array([1279,1664,1393,1631,1883,2351,2181,2678,2922,2894,2864,2630,3101]) #temp day highest
tnight = np.array([979,984,1015,792,1098,1267,986,1507,1128,815,1528,1256,1776]) #temp night lowest in kelv
m = np.array([1.142,2.052,0.69,3.31,0.357,7.341,1.114,1.741,27.23,10.4296,1.49,1.47,2.1])  #mass, jupiter mass
r = np.array([1.138,1.036,1.38,1.465,0.718,1.281,1.395,1.431,1.15,1.165,1.528,1.9,1.603]) #radius of planet
rstar = np.array([0.805,0.667,1.203,0.902,1.497,1.306,1.004,2.0,1.471,1.23,1.436,1.657,1.444]) #stellar radius
